План по работе над проектом:
* Ознакомление с датафреймом ga_hits и описаниями представленных атрибутов.
* Создание целевой переменной.

* Ознакомление с датафреймом ga_sessions и описаниями представленных атрибутов.
* оценка чистоты и полноты данных.
* Оброботка пустых значении.
* Проведение базовой чистки (дубликаты, пустые значения, типизация данных, ненужные атрибуты).
* Объеденение датафреймов

* Создание пайплайна с нужными шагами для корректного обучения модели.
* Обучение моделей.
* Сравнение моделей.

Импорт нужных библеотек

In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer

Первым делом разберемся с целевой переменной, которую нужно взять из файла ga_hits. Напоминание - целевое действие это любое совершенное событие типа "Оставить заявку" и "Заказать звонок"

In [2]:
data1 = pd.read_csv('ga_hits.csv')

In [3]:
data1 = pd.DataFrame(data1)
df=data1.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15726470 entries, 0 to 15726469
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   hit_date        object 
 2   hit_time        float64
 3   hit_number      int64  
 4   hit_type        object 
 5   hit_referer     object 
 6   hit_page_path   object 
 7   event_category  object 
 8   event_action    object 
 9   event_label     object 
 10  event_value     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 1.3+ GB


In [5]:
df.shape

(15726470, 11)

In [6]:
df  

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15726465,6866159858916559617.1640270865.1640270865,2021-12-23,810589.0,43,event,NaN,sberauto.com/cars/all/toyota/fortuner/24cb5af2...,quiz,quiz_show,NaN,NaN
15726466,7310304587364460692.1640261783.1640261783,2021-12-23,904927.0,40,event,NaN,sberauto.com/cars/all/mercedes-benz/gla-klasse...,quiz,quiz_show,NaN,NaN
15726467,8013702685784312179.1640270195.1640270195,2021-12-23,2172865.0,43,event,NaN,sberauto.com/cars/all/toyota/alphard/2ebe4871?...,quiz,quiz_show,NaN,NaN
15726468,8021505554734405918.1640257821.1640257821,2021-12-23,713325.0,45,event,NaN,sberauto.com/cars/all/bmw/x3/6a660f0a?rental_p...,quiz,quiz_show,NaN,NaN


Анализируем действия. Видим что просмотров карточки больше всего. (оно и логично для этого и заходят на сайт скорее всего)

In [7]:
df['event_action'].value_counts().sort_values(ascending=False)

event_action
view_card                     3558985
view_new_card                 3551009
sub_landing                   1798117
go_to_car_card                 973666
sub_view_cars_click            791515
                               ...   
click_on_android                    1
new_year_road_help_btn              1
titul_page_view                     1
show_pos_status_declined            1
survey_buy_after_sell_when          1
Name: count, Length: 230, dtype: int64

Создаем сначала колонку таргет который принимает 1 если присудствует целевое действие, и 0 если целевого действия нет. Затем создаем новый датафрейм сгрупированный по уникальным значениям session_id, с колонкой таргет который принимает максимальное значение оставляя лишь уникальные. Далее используем ресет индекс что бы сбросить индексы

In [8]:
df['target'] = df['event_action'].isin(['sub_car_claim_click', 'sub_car_claim_submit_click', 'sub_open_dialog_click', 'sub_custom_question_submit_click', 'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success', 'sub_car_request_submit_click']).astype(int)

df_target = df.groupby('session_id')['target'].max().reset_index()

Проверяем нет ли ошибок

In [9]:
df[df['event_action'].isin(['sub_car_claim_click', 'sub_car_claim_submit_click', 'sub_open_dialog_click', 'sub_custom_question_submit_click', 'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success', 'sub_car_request_submit_click'])]

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value,target
4016,2744563715298057088.1640258436.1640258436,2021-12-23,843092.0,81,event,NaN,sberauto.com/cars/all/kia/rio/fee33fe6?utm_sou...,sub_submit,sub_submit_success,nsPPIRqjxBefONGPpnsF,NaN,1
4045,3087297479839089634.1640268774.1640268774,2021-12-23,194144.0,22,event,NaN,sberauto.com/cars/all/skoda/rapid/bf24b977?utm...,sub_submit,sub_submit_success,nsPPIRqjxBefONGPpnsF,NaN,1
4046,3156966333326004302.1640206419.1640206800,2021-12-23,327223.0,63,event,NaN,sberauto.com/cars/all/skoda/rapid/bf24b977?utm...,sub_submit,sub_submit_success,nsPPIRqjxBefONGPpnsF,NaN,1
4047,3750243879753098158.1640272208.1640272208,2021-12-23,156992.0,20,event,NaN,sberauto.com/cars/all/nissan/x-trail/0744675f?...,sub_submit,sub_submit_success,nsPPIRqjxBefONGPpnsF,NaN,1
4048,7518333712042258254.1640258901.1640258901,2021-12-23,170616.0,16,event,NaN,sberauto.com/cars/all/mercedes-benz/gla-klasse...,sub_submit,sub_submit_success,KuMiABMMbspIDDhiCNVS,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
15725025,1277864870843199549.1636773954.1636773954,2021-11-13,354017.0,30,event,NaN,sberauto.com/cars/all/lada-vaz/vesta/2fc745ed?...,sub_button_click,sub_car_claim_submit_click,NaN,NaN,1
15725133,965861352491898977.1636751459.1636751459,2021-11-13,172590.0,18,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,sub_submit,sub_submit_success,nsPPIRqjxBefONGPpnsF,NaN,1
15725134,4727705172767122620.1636818847.1636818847,2021-11-13,356120.0,43,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,sub_submit,sub_submit_success,uimgZZmhfLQwbKAZZfCk,NaN,1
15725135,4727705172767122620.1636818847.1636818847,2021-11-13,312303.0,41,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,sub_button_click,sub_open_dialog_click,ZaZuwAXOKlbzyhUqtnmk,NaN,1


In [10]:
df_target

,session_id,target
0,1000009318903347362.1632663668.1632663668,0
1,1000010177899156286.1635013443.1635013443,0
2,1000013386240115915.1635402956.1635402956,0
3,1000017303238376207.1623489300.1623489300,0
4,1000020580299877109.1624943350.1624943350,0
...,...,...
1734605,999960188766601545.1626816843.1626816843,0
1734606,99996598443387715.1626811203.1626811203,0
1734607,999966717128502952.1638428330.1638428330,0
1734608,999988617151873171.1623556243.1623556243,0


In [11]:
df['session_id'].value_counts().sort_values(ascending=False)

session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
                                            ... 
160826705592749328.1631620368.1631620368       1
1855401207506421650.1631602579.1631602579      1
1481604624575304990.1633883424.1633883424      1
3751290052173382050.1631601058.1631601058      1
5381267721977892188.1640271196.1640271196      1
Name: count, Length: 1734610, dtype: int64

Далее работаем со вторым файлом, по которому нужно будет предсказывать целевую переменную

In [7]:
data2 = pd.read_csv('ga_sessions.csv', low_memory=False)

In [8]:
data2 = pd.DataFrame(data2)
df2 = data2.copy()

DataSpell показывает все пропуски и соотношения данных(но так как я в первые им пользуюсь дальше я хочу удостовериться что показывается все как надо)

In [9]:
df2

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,NaN,NaN,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,Chrome,Russia,Chelyabinsk


In [15]:
df2[df2['utm_source'].isna()]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
77652,1100240274910044288.1640728207.1640728207,256169651.1637310592,2021-12-29,00:50:07,2,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,OnePlus,NaN,412x892,Chrome,Russia,Saint Petersburg
122841,1303431342277938317.1637250145.1637250145,303478758.1637239949,2021-11-18,18:42:25,3,NaN,Sbol_catalog,cccMlyVfjXspfaCSrMsO,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Balashikha
136220,1364914954463538089.1637595059.1637595059,317794027.1634397097,2021-11-22,18:30:59,8,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,YaBrowser,Russia,Moscow
136221,1364914954463538089.1638478637.1638478637,317794027.1634397097,2021-12-02,23:57:17,9,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,YaBrowser,Russia,Moscow
139706,1380043242326371104.1637518268.1637518268,321316356.1636477728,2021-11-21,21:11:08,4,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,360x740,Chrome,Russia,Izhevsk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629120,8025307339522195714.1638678933.1638678933,1868537473.1636712706,2021-12-05,07:35:33,13,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Sochi
1629121,8025307339522195714.1638688249.1638688249,1868537473.1636712706,2021-12-05,10:10:49,14,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Krasnodar
1629122,8025307339522195714.1638954236.1638954236,1868537473.1636712706,2021-12-08,12:03:56,15,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x780,Chrome,Ukraine,Pereval's'k
1732460,848873006059532776.1637459623.1637459623,197643648.1637396968,2021-11-21,04:53:43,2,NaN,(not set),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,360x800,Chrome,Russia,Novosibirsk


Пишем функцию которая заполняет target_column значениями которые наипопулярны для заполненных строк при тех же комбинациях col1 и col2

In [17]:
def fill_missing_values_optimized(df, target_column, col1, col2):

# Работаем с копией данных на случай ошибок в коде
    df_copy = df.copy()

# Создаем булевую серию, которая содержит True для NaN значений в целевой колонке
    missing_rows = df_copy[target_column].isnull()

# Создаем пустой словарь mode_dict, который будет использоваться для хранения рассчитанных значений моды для соответствующих комбинаций значений из столбцов 'col1' и 'col2'
    mode_dict = {}

# Первый цикл for итерируется по каждой строке DataFrame. Если в текущей строке нет пропущенного значения в 'target_column', а значения в 'col1' и 'col2' не являются NaN, то создается кортеж key из значений в 'col1' и 'col2'. Затем проверяется, есть ли такой ключ в mode_dict, и если нет, то рассчитывается мода для всех строк, у которых значения в 'col1' и 'col2' совпадают с текущей строкой, и результат добавляется в словарь
    for i in range(df_copy.shape[0]):
        if not missing_rows[i] and not pd.isna(df_copy.loc[i, col1]) and not pd.isna(df_copy.loc[i, col2]):
            key = (df_copy.loc[i, col1], df_copy.loc[i, col2])
            if key not in mode_dict:
                matching_rows = df_copy.loc[(df_copy[col1] == df_copy.loc[i, col1]) &
                                            (df_copy[col2] == df_copy.loc[i, col2])]
                mode_dict[key] = matching_rows[target_column].mode()


# Второй цикл for также итерируется по каждой строке DataFrame. Если в текущей строке есть пропущенное значение в 'target_column', и значения в 'col1' и 'col2' не являются NaN, то создается кортеж key из значений в 'col1' и 'col2'. Затем проверяется, есть ли такой ключ в mode_dict, и если есть, то пропущенное значение в 'target_column' заменяется на рассчитанное значение моды 
    for i in range(df_copy.shape[0]):
        if missing_rows[i] and not pd.isna(df_copy.loc[i, col1]) and not pd.isna(df_copy.loc[i, col2]):
            key = (df_copy.loc[i, col1], df_copy.loc[i, col2])
            mode = mode_dict.get(key)
            if mode is not None and len(mode) > 0:
                df_copy.loc[i, target_column] = mode[0]

    return df_copy

Далее очень старательно используем созданную функцию, что бы максимально не лишаться данных удалая или просто заменяя NaN значения

In [17]:
df2_copy = df2.copy()

In [18]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_source', 'utm_campaign', 'utm_keyword')

In [19]:
df2_copy

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,NaN,NaN,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,Chrome,Russia,Chelyabinsk


In [20]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_campaign', 'utm_source', 'utm_keyword')

In [21]:
df2_copy

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,NaN,NaN,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,Chrome,Russia,Chelyabinsk


In [22]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_campaign', 'utm_source', 'utm_adcontent')

In [23]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_adcontent', 'utm_source', 'utm_campaign')

In [24]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_adcontent', 'utm_source', 'utm_keyword')

In [25]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_keyword', 'utm_source', 'utm_campaign')

In [26]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_keyword', 'utm_source', 'utm_adcontent')

In [27]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_keyword', 'utm_source', 'utm_medium')

In [28]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_adcontent', 'utm_source', 'utm_medium')

In [29]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_campaign', 'utm_source', 'utm_medium')

Что бы работать с колонками device более оптимально добавляем еще одну колонку(потому что видим еще 3 полностью заполненных колонок)

In [18]:
def fill_missing_values_optimized_device(df, target_column, col1, col2, col3):

    df_copy = df.copy()

    missing_rows = df_copy[target_column].isnull()

    mode_dict = {}
    for i in range(df_copy.shape[0]):
        if not missing_rows[i] and not pd.isna(df_copy.loc[i, col1]) and not pd.isna(df_copy.loc[i, col2]) and not pd.isna(df_copy.loc[i, col3]):
            key = (df_copy.loc[i, col1], df_copy.loc[i, col2], df_copy.loc[i, col3])
            if key not in mode_dict:
                matching_rows = df_copy.loc[(df_copy[col1] == df_copy.loc[i, col1]) &
                                            (df_copy[col2] == df_copy.loc[i, col2]) &
                                            (df_copy[col3] == df_copy.loc[i, col3])]
                mode_dict[key] = matching_rows[target_column].mode()


    for i in range(df_copy.shape[0]):
                if missing_rows[i] and not pd.isna(df_copy.loc[i, col1]) and not pd.isna(df_copy.loc[i, col2]) and not pd.isna(df_copy.loc[i, col3]):
                    key = (df_copy.loc[i, col1], df_copy.loc[i, col2], df_copy.loc[i, col3])
                    mode = mode_dict.get(key)
                    if mode is not None and len(mode) > 0:
                        df_copy.loc[i, target_column] = mode[0]

    return df_copy

In [31]:
df2_copy = fill_missing_values_optimized_device(df2_copy, 'device_os', 'device_browser', 'device_category', 'device_screen_resolution')

In [32]:
df2_copy = fill_missing_values_optimized_device(df2_copy, 'device_brand', 'device_browser', 'device_category', 'device_screen_resolution')

In [33]:
df2_copy = fill_missing_values_optimized_device(df2_copy, 'device_model', 'device_browser', 'device_category', 'device_screen_resolution')

In [34]:
df2_copy = fill_missing_values_optimized_device(df2_copy, 'device_model', 'device_browser', 'device_os', 'device_screen_resolution')

In [35]:
df2_copy = fill_missing_values_optimized_device(df2_copy, 'device_brand', 'device_browser', 'device_os', 'device_screen_resolution')

In [36]:
df2_copy = fill_missing_values_optimized_device(df2_copy, 'device_model', 'device_brand', 'device_os', 'device_screen_resolution')

In [37]:
df2_copy = fill_missing_values_optimized(df2_copy, 'utm_keyword', 'utm_adcontent', 'utm_campaign')

In [38]:
df2_copy

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,fJCYsujgSxIHFbOmgDdN,NOBKLgtuvqYWkXQHeYWM,tzIeQUUHahhBuEkJqDVp,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,VlqBmecIOXWjCWUmQkLd,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Chelyabinsk


In [39]:
df2_copy = fill_missing_values_optimized(df2_copy, 'device_model', 'device_brand', 'device_screen_resolution')
df2_copy

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,fJCYsujgSxIHFbOmgDdN,NOBKLgtuvqYWkXQHeYWM,tzIeQUUHahhBuEkJqDVp,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,VlqBmecIOXWjCWUmQkLd,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Chelyabinsk


Даже после этого всего остаются колонки с NaN значениями. Почему? Потому что комбинации остальных колонок не имеют других значений кроме NaN. Полагаю сейчас мы сделали что могли. Дальше не тратим силы и начнем обьеденять датафреймы по колонке session_id и будем уже на нем решать оставшиеся проблемы

In [40]:
df_1 = df[['session_id', 'target']]

In [41]:
# Не правильно объеденил данные
df_fin = df_1.merge(df2_copy, on='session_id', how='inner')

In [45]:
df_fin

,session_id,target,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,5639623078712724064.1640254056.1640254056,0,1313077071.1640254048,2021-12-23,13:07:36,2,kjsLglQLzykiRbcDiGcD,cpc,NSbcaLfXREiOVOXTmbnD,JNHcPlZPxEMWDnRiyoBf,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Saint Petersburg
1,5639623078712724064.1640254056.1640254056,0,1313077071.1640254048,2021-12-23,13:07:36,2,kjsLglQLzykiRbcDiGcD,cpc,NSbcaLfXREiOVOXTmbnD,JNHcPlZPxEMWDnRiyoBf,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Saint Petersburg
2,5639623078712724064.1640254056.1640254056,0,1313077071.1640254048,2021-12-23,13:07:36,2,kjsLglQLzykiRbcDiGcD,cpc,NSbcaLfXREiOVOXTmbnD,JNHcPlZPxEMWDnRiyoBf,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Saint Petersburg
3,5639623078712724064.1640254056.1640254056,0,1313077071.1640254048,2021-12-23,13:07:36,2,kjsLglQLzykiRbcDiGcD,cpc,NSbcaLfXREiOVOXTmbnD,JNHcPlZPxEMWDnRiyoBf,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Saint Petersburg
4,5639623078712724064.1640254056.1640254056,0,1313077071.1640254048,2021-12-23,13:07:36,2,kjsLglQLzykiRbcDiGcD,cpc,NSbcaLfXREiOVOXTmbnD,JNHcPlZPxEMWDnRiyoBf,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Saint Petersburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15685214,1706097853564481669.1640267190.1640267190,0,397231861.1640263813,2021-12-23,16:46:30,2,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,428x926,Safari,Russia,Moscow
15685215,8262758806963127884.1640272536.1640272536,0,1923823451.1640269388,2021-12-23,18:15:36,2,IZEXUFLARCUMynmHNBGo,blogger_channel,foFTSdUvNqqkPzZvgiqt,DZYjhfIUfdqhfuTNUmjn,puhZPIYqKXeFPaUviSjo,mobile,Android,Realme,AuMdmADEIoPXiWpTsBEj,360x800,Chrome,Russia,Ulyanovsk
15685216,3349670872968620291.1640264206.1640264206,0,779906025.1640261891,2021-12-23,15:56:46,2,QxAxdyPLuQMEcrdZWdWb,cpa,mSVMBvlHFgftJaQAZEdp,WbmadpYgwtVCcAwGJoXb,gEydMgyogRBSsihrQQwd,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari (in-app),Russia,Samara
15685217,1009509786569589790.1640244938.1640244938,0,235044813.1640154142,2021-12-23,10:35:38,3,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,tVtbIKrPSOvrXLCznVVe,desktop,Windows,NaN,NaN,1366x768,Edge,Russia,Orenburg


Сохраняю данные что бы не делать все заново когда в очередной раз поймаю синий экран смерти

In [48]:
df_fin1 = df2_copy.merge(df_target, on='session_id', how='inner')

In [49]:
df_fin1.to_csv('save_df1')

Далее загружаем сохраненный файл и работаем с ним

In [2]:
data_cheak = pd.read_csv('save_df1', low_memory=False)

In [3]:
data_cheak = pd.DataFrame(data_cheak)
df_ch_copy = data_cheak.copy()

In [5]:
df_ch_copy

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust,0
1,1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow,0
2,2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk,0
3,3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,fJCYsujgSxIHFbOmgDdN,NOBKLgtuvqYWkXQHeYWM,tzIeQUUHahhBuEkJqDVp,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
4,4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732261,1732261,9055415581448263752.1640159305.1640159305,2.108378e+09,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,VlqBmecIOXWjCWUmQkLd,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow,0
1732262,1732262,9055421130527858185.1622007305.1622007305,2.108380e+09,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Stavropol,0
1732263,1732263,9055422955903931195.1636979515.1636979515,2.108380e+09,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow,0
1732264,1732264,905543020766873816.1638189404.1638189404,2.108382e+08,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Chelyabinsk,0


Удаляем дубликаты

In [6]:
df_ch_copy = df_ch_copy.drop_duplicates()

In [7]:
df_ch_copy

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust,0
1,1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow,0
2,2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk,0
3,3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,fJCYsujgSxIHFbOmgDdN,NOBKLgtuvqYWkXQHeYWM,tzIeQUUHahhBuEkJqDVp,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
4,4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732261,1732261,9055415581448263752.1640159305.1640159305,2.108378e+09,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,VlqBmecIOXWjCWUmQkLd,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow,0
1732262,1732262,9055421130527858185.1622007305.1622007305,2.108380e+09,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Stavropol,0
1732263,1732263,9055422955903931195.1636979515.1636979515,2.108380e+09,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow,0
1732264,1732264,905543020766873816.1638189404.1638189404,2.108382e+08,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Chelyabinsk,0


Выделяем из датасета только строки со значение 0 в колонке таргет

In [8]:
df_filtered = df_ch_copy.loc[df_ch_copy['target']==0]

In [9]:
df_filtered

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust,0
1,1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow,0
2,2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk,0
3,3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,fJCYsujgSxIHFbOmgDdN,NOBKLgtuvqYWkXQHeYWM,tzIeQUUHahhBuEkJqDVp,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
4,4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732261,1732261,9055415581448263752.1640159305.1640159305,2.108378e+09,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,VlqBmecIOXWjCWUmQkLd,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow,0
1732262,1732262,9055421130527858185.1622007305.1622007305,2.108380e+09,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Stavropol,0
1732263,1732263,9055422955903931195.1636979515.1636979515,2.108380e+09,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow,0
1732264,1732264,905543020766873816.1638189404.1638189404,2.108382e+08,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Chelyabinsk,0


Строк где таргет равен нулю 1681952, так что без сожелении удалям строки с NaN значениями

In [10]:
df_filtered = df_filtered.dropna()

In [11]:
df_filtered

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Zlatoust,0
1,1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Samsung Internet,Russia,Moscow,0
2,2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Krasnoyarsk,0
3,3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,fJCYsujgSxIHFbOmgDdN,NOBKLgtuvqYWkXQHeYWM,tzIeQUUHahhBuEkJqDVp,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
4,4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732260,1732260,9055398929844789828.1624891784.1624891784,2.108374e+09,2021-06-28,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,vXsFkagGabkcWKlgLzSg,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Samsung,FTAuYVNoYYxgvKMpKSLW,320x676,Chrome,Russia,Naro-Fominsk,0
1732261,1732261,9055415581448263752.1640159305.1640159305,2.108378e+09,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,VlqBmecIOXWjCWUmQkLd,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Moscow,0
1732262,1732262,9055421130527858185.1622007305.1622007305,2.108380e+09,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Safari,Russia,Stavropol,0
1732263,1732263,9055422955903931195.1636979515.1636979515,2.108380e+09,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,375x667,Safari,Russia,Moscow,0


Все еще слишком много колонок для моего старого ноутбука, удаляем рандомные 70% данных

In [12]:
random_rows = df_filtered.sample(frac=0.7, random_state=42)

In [13]:
df_filtered = df_filtered.drop(random_rows.index)

In [14]:
df_filtered

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
4,4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x786,Chrome,Russia,Moscow,0
9,9,9055466554104774132.1624800757.1624800757,2.108390e+09,2021-06-27,16:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow,0
19,19,9055491722626447422.1638119489.1638119489,2.108396e+09,2021-11-28,20:11:29,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Saint Petersburg,0
20,20,9055496369776380607.1633438303.1633438303,2.108397e+09,2021-10-05,15:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Novorossiysk,0
23,23,9055497958914887309.1634048072.1634048072,2.108397e+09,2021-10-12,17:00:00,3,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,pHqgLcdBQPSLcrTpLkdr,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x851,Chrome,Russia,Moscow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732233,1732233,9055283330814105899.1638979884.1638979890,2.108347e+09,2021-12-08,19:11:30,1,bByPQxmDaMXgpHeypKSM,referral,QdLfySaGXolfTBSNVfHn,SOkCdPxfUcZUzzOdgGES,puhZPIYqKXeFPaUviSjo,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Krasnodar,0
1732234,1732234,9055290018073398651.1634192763.1634192763,2.108349e+09,2021-10-14,09:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Kurgan,0
1732235,1732235,9055300712540178759.1632405870.1632405870,2.108351e+09,2021-09-23,17:04:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,zPJpddwzkFqLMSYgtDqy,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,385x854,Chrome,Russia,Moscow,0
1732260,1732260,9055398929844789828.1624891784.1624891784,2.108374e+09,2021-06-28,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,vXsFkagGabkcWKlgLzSg,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Samsung,FTAuYVNoYYxgvKMpKSLW,320x676,Chrome,Russia,Naro-Fominsk,0


Выделяем строки где колонка таргет равен 1

In [15]:
df_filtered_target = df_ch_copy.loc[df_ch_copy['target']==1]

In [16]:
df_filtered_target

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
69,69,9055768928391199041.1623652678.1623652678,2.108461e+09,2021-06-14,09:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Moscow,1
73,73,9055789093275396095.1636395007.1636395007,2.108465e+09,2021-11-08,21:10:07,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,414x736,Safari,Russia,Samara,1
76,76,9055794766915104430.1624305327.1624305327,2.108467e+09,2021-06-21,22:00:00,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,ZNhSIPchlbnKwZWcsKzz,desktop,Macintosh,NaN,NaN,834x1112,Safari,Russia,Domodedovo,1
127,127,9056028189803556573.1630154471.1630154471,2.108521e+09,2021-08-28,15:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,jXFxTkHxuiBKWOvUtsIa,412x915,YaBrowser,Russia,Krasnodar,1
135,135,9056093322981390023.1628944098.1628944098,2.108536e+09,2021-08-14,15:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x740,Chrome,Russia,Moscow,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732152,1732152,9054912872702758664.1636781836.1636781836,2.108261e+09,2021-11-13,08:37:16,1,kjsLglQLzykiRbcDiGcD,cpc,LwJZxKuWmvOhPsaCeRjG,JNHcPlZPxEMWDnRiyoBf,HKFjmDlYppkKqsWPogJt,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,360x800,Chrome,Russia,Moscow,1
1732225,1732225,9055248417020050011.1634073188.1634073188,2.108339e+09,2021-10-13,00:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,SgIUDYUKnyWHVowUOqid,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Nizhny Novgorod,1
1732227,1732227,9055248417020050011.1634075541.1634075541,2.108339e+09,2021-10-13,00:00:00,3,ZpYIoDJMcFzVoPFsHGJL,banner,SgIUDYUKnyWHVowUOqid,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Nizhny Novgorod,1
1732254,1732254,9055376699099939975.1630766214.1630766214,2.108369e+09,2021-09-04,17:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Khimki,1


Заполняем пустоты модой используя SimpleImputer

In [17]:
imputer = SimpleImputer(strategy='most_frequent')

In [18]:
columns = list(df_filtered_target.columns)

In [19]:
df_filtered_target = imputer.fit_transform(df_filtered_target[columns])

In [20]:
df_filtered_target = pd.DataFrame(df_filtered_target, columns=columns)

In [21]:
df_filtered_target

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,69,9055768928391199041.1623652678.1623652678,2108460508.162365,2021-06-14,09:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Moscow,1
1,73,9055789093275396095.1636395007.1636395007,2108465203.16364,2021-11-08,21:10:07,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,414x736,Safari,Russia,Samara,1
2,76,9055794766915104430.1624305327.1624305327,2108466524.16243,2021-06-21,22:00:00,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,ZNhSIPchlbnKwZWcsKzz,desktop,Macintosh,Apple,AuMdmADEIoPXiWpTsBEj,834x1112,Safari,Russia,Domodedovo,1
3,127,9056028189803556573.1630154471.1630154471,2108520872.163016,2021-08-28,15:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,jXFxTkHxuiBKWOvUtsIa,412x915,YaBrowser,Russia,Krasnodar,1
4,135,9056093322981390023.1628944098.1628944098,2108536037.162894,2021-08-14,15:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x740,Chrome,Russia,Moscow,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50309,1732152,9054912872702758664.1636781836.1636781836,2108261192.163678,2021-11-13,08:37:16,1,kjsLglQLzykiRbcDiGcD,cpc,LwJZxKuWmvOhPsaCeRjG,JNHcPlZPxEMWDnRiyoBf,HKFjmDlYppkKqsWPogJt,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,360x800,Chrome,Russia,Moscow,1
50310,1732225,9055248417020050011.1634073188.1634073188,2108339317.163408,2021-10-13,00:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,SgIUDYUKnyWHVowUOqid,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Nizhny Novgorod,1
50311,1732227,9055248417020050011.1634075541.1634075541,2108339317.163408,2021-10-13,00:00:00,3,ZpYIoDJMcFzVoPFsHGJL,banner,SgIUDYUKnyWHVowUOqid,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Nizhny Novgorod,1
50312,1732254,9055376699099939975.1630766214.1630766214,2108369185.163077,2021-09-04,17:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Khimki,1


Объеденяем в финальный даатфрейм


In [22]:
final_data = pd.concat([df_filtered_target, df_filtered])

In [23]:
final_data = final_data.sample(frac=1)

In [24]:
final_data

,Unnamed: 0,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
47680,1640352,861567602239642895.1635425553.1635425553,200599339.163543,2021-10-28,15:52:33,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,UbQpozKfTgYxQovHjkQM,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow,1
48209,1658798,8703059496721564670.1630389395.1630389395,2026338944.163039,2021-08-31,08:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,dUuXlWzvmhDSyclWRhNP,pHqgLcdBQPSLcrTpLkdr,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Samsung Internet,Russia,Kubinka,1
961585,961585,5366140572278903224.1635701148.1635701148,1249401963.16357,2021-10-31,20:25:48,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Veliky Novgorod,0
959142,959142,5354870681162512304.1624802278.1624802278,1246777987.16248,2021-06-27,16:00:00,2,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Kirov,0
520782,520782,325238689346041728.1638872959.1638872959,75725533.163887,2021-12-07,13:29:19,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Saransk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290905,1290905,6944657193759066052.1625213471.1625213471,1616929004.162521,2021-07-02,11:00:00,4,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Chrome,Russia,Khimki,0
51445,51445,998273288816685235.1622012085.1622012085,232428612.162201,2021-05-26,09:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,ZkRlaMsFhSBPslXBtfiG,412x869,Chrome,Russia,Moscow,0
1031380,1031380,5699137684613566500.1634668580.1634668678,1326933895.163467,2021-10-19,21:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Volgograd,0
1654010,1654010,8680929029191131182.1632657455.1632657455,2021186293.163266,2021-09-26,14:00:00,1,RmEBuqrriAfAVsLQQmhk,cpc,KgicpPxiEQfzPlPwQZJq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Vivo,AuMdmADEIoPXiWpTsBEj,393x876,Chrome,Russia,Saint Petersburg,0


Удаляем колонку которая когда то была индексом

In [25]:
final_data = final_data.drop('Unnamed: 0', axis=1)

In [26]:
final_data

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
47680,861567602239642895.1635425553.1635425553,200599339.163543,2021-10-28,15:52:33,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,UbQpozKfTgYxQovHjkQM,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow,1
48209,8703059496721564670.1630389395.1630389395,2026338944.163039,2021-08-31,08:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,dUuXlWzvmhDSyclWRhNP,pHqgLcdBQPSLcrTpLkdr,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Samsung Internet,Russia,Kubinka,1
961585,5366140572278903224.1635701148.1635701148,1249401963.16357,2021-10-31,20:25:48,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Veliky Novgorod,0
959142,5354870681162512304.1624802278.1624802278,1246777987.16248,2021-06-27,16:00:00,2,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Kirov,0
520782,325238689346041728.1638872959.1638872959,75725533.163887,2021-12-07,13:29:19,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Saransk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290905,6944657193759066052.1625213471.1625213471,1616929004.162521,2021-07-02,11:00:00,4,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Chrome,Russia,Khimki,0
51445,998273288816685235.1622012085.1622012085,232428612.162201,2021-05-26,09:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,ZkRlaMsFhSBPslXBtfiG,412x869,Chrome,Russia,Moscow,0
1031380,5699137684613566500.1634668580.1634668678,1326933895.163467,2021-10-19,21:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Volgograd,0
1654010,8680929029191131182.1632657455.1632657455,2021186293.163266,2021-09-26,14:00:00,1,RmEBuqrriAfAVsLQQmhk,cpc,KgicpPxiEQfzPlPwQZJq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Vivo,AuMdmADEIoPXiWpTsBEj,393x876,Chrome,Russia,Saint Petersburg,0


In [34]:
missing_values = final_data.isnull().sum()

In [35]:
missing_values

session_id                  0
client_id                   0
visit_date                  0
visit_time                  0
visit_number                0
utm_source                  0
utm_medium                  0
utm_campaign                0
utm_adcontent               0
utm_keyword                 0
device_category             0
device_os                   0
device_brand                0
device_model                0
device_screen_resolution    0
device_browser              0
geo_country                 0
geo_city                    0
target                      0
dtype: int64

Сохраняем и радуемся

In [36]:
final_data.to_csv('final_data', index=False)

In [4]:
df = pd.read_csv('final_data')

In [5]:
data2

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,NaN,NaN,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,Chrome,Russia,Chelyabinsk


In [6]:
df

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,861567602239642895.1635425553.1635425553,2.005993e+08,2021-10-28,15:52:33,1,BHcvLfOaCWvWTykYqHVe,cpc,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,UbQpozKfTgYxQovHjkQM,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Moscow,1
1,8703059496721564670.1630389395.1630389395,2.026339e+09,2021-08-31,08:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,dUuXlWzvmhDSyclWRhNP,pHqgLcdBQPSLcrTpLkdr,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Samsung Internet,Russia,Kubinka,1
2,5366140572278903224.1635701148.1635701148,1.249402e+09,2021-10-31,20:25:48,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,AuMdmADEIoPXiWpTsBEj,360x640,Chrome,Russia,Veliky Novgorod,0
3,5354870681162512304.1624802278.1624802278,1.246778e+09,2021-06-27,16:00:00,2,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,Samsung,AuMdmADEIoPXiWpTsBEj,1920x1080,Chrome,Russia,Kirov,0
4,325238689346041728.1638872959.1638872959,7.572553e+07,2021-12-07,13:29:19,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,AuMdmADEIoPXiWpTsBEj,360x720,Chrome,Russia,Saransk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434912,6944657193759066052.1625213471.1625213471,1.616929e+09,2021-07-02,11:00:00,4,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,iOS,Apple,AuMdmADEIoPXiWpTsBEj,390x844,Chrome,Russia,Khimki,0
434913,998273288816685235.1622012085.1622012085,2.324286e+08,2021-05-26,09:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,ZkRlaMsFhSBPslXBtfiG,412x869,Chrome,Russia,Moscow,0
434914,5699137684613566500.1634668580.1634668678,1.326934e+09,2021-10-19,21:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Volgograd,0
434915,8680929029191131182.1632657455.1632657455,2.021186e+09,2021-09-26,14:00:00,1,RmEBuqrriAfAVsLQQmhk,cpc,KgicpPxiEQfzPlPwQZJq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Vivo,AuMdmADEIoPXiWpTsBEj,393x876,Chrome,Russia,Saint Petersburg,0


Какой я вывод могу сделать после проделанной работы над датафреймами. В данных огромнейший дисбаланс. Слишком много данных я потерял пытаясь достичь того что модель будет получать хоть несколько сотен значений таргет не равный нулю. Никак не смог понять закодированные колонки ['utm_source', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_model'], но не могу их удалять потому что модель может их понять. Что касается пустых значении - в среднем их очень много, есть даже колонка которая состоит из пропуском на 99,1%(1843704 значении). По итогу удалось хоть и не идеально но заполнить данные и получить не плохой результат, но думается мне что если бы колонки не были бы закодированы то мы бы смогли получить гораздо больше информации по этим данным, и даже модели было бы легче с ними совладать.